In [12]:
# Install required packages
%pip install senml
%pip install zenml
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2  # required for calab

# Restart the kernel to apply changes
#import IPython
#IPython.Application.instance().kernel.do_shutdown(restart=True)


⠇ Installing integrations...


In [13]:
#%pip install 'senml[server]'
#!zenml integration install sklearn -y
#%pip install pyparsing==2.4.2 #required for calab

#import IPython


# Restart the kernel to apply changes
import IPython
IPython.Application.instance().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [14]:
#use ngrok accoun tto view some visualiztaions later for this.


NGROK_TOKEN = '30FylZrtlgkNeOhPKjBPmWEjVV4_87LekmMer9ZUhe45i5J4z'
# per tutorial 2SqcaHGFW0j4yB9afavkFqHR7Jy_22bRJ3oDDA8LfGKdkKMHo
#mine is 30FylZrtlgkNeOhPKjBPmWEjVV4_87LekmMer9ZUhe45i5J4z

In [15]:
from zenml.environment import Environment

if Environment.in_google_colab(): #colab only setup
    #install and authenticate ngrok
    !pip install pyngrok
    !ngrok authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [16]:
# Reinstall ZenML integration for sklearn after restart
#!zenml integration install sklearn -y

ZenML Setup

ZenML is tool for this task is intuitive and has integrations with most of the advanced MLOps tools we shall use. NB: have ZenML isntealled via(pip install zenml)

In [17]:
!rm -rf .zen
!zenml init

⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠇ Initiali

Example Experimentation ML CODE
using SVC (Bad but quick example)

In [1]:
from os import X_OK
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split


def train_test() -> None:
  """Train and test a scikit learn svc classifier on digits"""
  digits = load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.25, shuffle=False)
  model = SVC(gamma = 0.0001)
  model.fit(X_train, y_train)
  test_acc = model.score(X_test, y_test)
  print(f"Test accuracy: {test_acc}")

train_test()

Test accuracy: 0.9288888888888889


Turning Experiment into ML Pipline with ZenML

In practice ML workflows will be more complicated. THere might be preprocessing that you dont want to redo every time you train a model, you will need to compare the performance of different modesl, deploy them in a dproduction setting, and more. ML pipelines allow us to define our workflows in modular steps that we can mix and match.

Importer -> SVC Trainer -> Evaluator

3 unique steps in this example: data loading, model training, model evaluation

We define each as a ZenML Pipeline Step by moving each step to its own function and decorating them with ZenML's @step Python decorator

In [2]:
from re import X
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer () -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]: #importer will load digits and load data and return, we have to show this bc trainner needs to know what type of stuff is imported and sending

    "load the digits dataset as nump arrays "
    digits =load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.25, shuffle=False)
    return X_train, X_test, y_train, y_test

@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    "train an sklearn svc classifier"
    model = SVC(gamma=0.0001)
    model.fit(X_train, y_train)
    return model

@step
def evaluator(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin,
) -> float:
    "calculate the accuracy score of the model"
    test_acc = model.score(X_test, y_test)
    print(f"Test accuracy: {test_acc}")
    return test_acc


Connecting Steps

Use @pipeline decorator to connect all of our steps into ML pipeline

The pipeline definition doesn ot depend on the ocntrete step functions as previously defined above; it just establishes a recipe for how data moves through the steps. This means we can replace steps as needed. EX) to run the same pipeline with diff models to compare their performances

In [3]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  "links al lthe steps together in a pipelline"
  X_train, X_test, y_train, y_test = importer()
  model = svc_trainer(X_train = X_train, y_train = y_train)
  evaluator(X_test = X_test, y_test = y_test, model = model)


Run ZenML Pipelines

Initialize pipeline with concrete step functions and call run() method to run it

In [4]:
digits_svc_pipeline = digits_pipeline()
#digits_svc_pipeline.run(unlisted = TRUE)

Initiating a new run for the pipeline: digits_pipeline.
In a future release, the default Python package installer used by ZenML to build container images for your containerized pipelines will change from 'pip' to 'uv'. To maintain current behavior, you can explicitly set python_package_installer=PythonPackageInstaller.PIP in your DockerSettings.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Using cached version of step importer.
Using cached version of step svc_trainer.
Using cached version of step evaluator.
All steps of the pipeline run were cached.


Visulize pipeline run in ZenML's dashboard

To run this, zenml up to spin up a ZenML dashboard locally, log in with username default and empty password, navigate to the "Runs: tabe in the "Pipelines" section

In [5]:
from zenml.environment import Environment

def start_zenml_dashboard(port = 8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok

    public_url = ngrok.connect(port)
    #print(f"Dashboard is available at {public_url}")
    print(F"\x1b[31mIn Colab, use this URL instead: {public_url}!\x1b[0m")
    !zenml up --blocking --port {port}
  else:
    !zenml up --blocking --port {port}
    #print(f"Dashboard is available at http://localhost:{port}

start_zenml_dashboard()



INFO:pyngrok.ngrok:Opening tunnel named: http-8237-2b2a3517-55ee-4094-8e9a-58f01cda387c


Opening tunnel named: http-8237-2b2a3517-55ee-4094-8e9a-58f01cda387c


INFO:pyngrok.process.ngrok:t=2025-07-23T04:45:27+0000 lvl=info msg="no configuration paths supplied"


t=2025-07-23T04:45:27+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2025-07-23T04:45:27+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2025-07-23T04:45:27+0000 lvl=crit msg="command failed" err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command ngrok start --all.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"


t=2025-07-23T04:45:27+0000 lvl=warn msg="failed to check for update" obj=updater err="Post \"https://update.equinox.io/check\": context canceled"


INFO:pyngrok.process.ngrok:t=2025-07-23T04:45:27+0000 lvl=info msg="no more state changes" obj=tunnels.session


t=2025-07-23T04:45:27+0000 lvl=info msg="no more state changes" obj=tunnels.session


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:15                                                                             │
│                                                                                                  │
│   12 │   get_ipython().system('zenml up --blocking --port {port}')                               │
│   13 │   #print(f"Dashboard is available at http://localhost:{port}                              │
│   14                                                                                             │
│ ❱ 15 start_zenml_dashboard()                                                                     │
│   16                                                                                             │
│   17                                                                                             │
│                                                                                                  │
│ in start_zenml_dashboard:7                                                                       │
│                                                                                                  │
│    4   if Environment.in_google_colab():                                                         │
│    5 │   from pyngrok import ngrok                                                               │
│    6 │                                                                                           │
│ ❱  7 │   public_url = ngrok.connect(port)                                                        │
│    8 │   #print(f"Dashboard is available at {public_url}")                                       │
│    9 │   print(F"\x1bIn Colab, use this URL instead: {public_url}!\x1b")                  │
│   10 │   get_ipython().system('zenml up --blocking --port {port}')                               │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/pyngrok/ngrok.py:385 in connect                          │
│                                                                                                  │
│   382 │                                                                                          │
│   383 │   logger.info(f"Opening tunnel named: {name}")                                           │
│   384 │                                                                                          │
│ ❱ 385 │   api_url = get_ngrok_process(pyngrok_config).api_url                                    │
│   386 │                                                                                          │
│   387 │   logger.debug(f"Creating tunnel with options: {options}")                               │
│   388                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/pyngrok/ngrok.py:203 in get_ngrok_process                │
│                                                                                                  │
│   200 │                                                                                          │
│   201 │   install_ngrok(pyngrok_config)                                                          │
│   202 │                                                                                          │
│ ❱ 203 │   return process.get_process(pyngrok_config)                                             │
│   204                                                                                            │
│   205                                                                                            │
│   206 def _apply_edge_to_tunnel(tunnel: NgrokTunnel,                                             │
│                                                                   